### 使用 request & bs4

In [ ]:
import os
import re
import random
import requests
from bs4 import BeautifulSoup as bs


Link = 'https://www.gutenberg.org'
chineseBooksPageURL = f'{Link}/browse/languages/zh'

folderPath = 'gutenberg_books'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

titleList = []
sameName_Key = 1

def GetAllBooksURL():
    res = requests.get(chineseBooksPageURL)
    soup = bs(res.text,'lxml')

    bookList = []
    booksName = soup.select('li > a[href]')
    
    for info in booksName:
        # 砍掉前後空白
        booksTitle = info.text.strip()
        # 中文正規表達式
        chineseRegex = r'[\u4e00-\u9fff]'

        # 若有中文則加入書單
        if re.search(chineseRegex,booksTitle) != None:
            bookList.append(
                {
                    "Title" : booksTitle,
                    "URL" : Link + info['href']
                }
            )
    
    return bookList

def getBooksInfo(book):
    global sameName_Key
    # 取得書本網頁
    res = requests.get(book["URL"])
    soup = bs(res.text,'lxml')

    # 取得書本內文連結
    bookLink = soup.find('a', text = 'Plain Text UTF-8')

    downloadLink = Link + bookLink['href']
    bookInfo = requests.get(downloadLink)

    # 避免windows存檔及部分特殊字元
    specialText = r'[<>:\"/\\|?*\—\-=+\~`\']+|\s+'
    file_title = re.sub(specialText, "_", book['Title'])

    # 若檔名相同，給與一個隨機值附加在檔名
    if file_title in titleList:
        # 給 100~999 降低重複機率
        sameName_Key = random.randint(100,1000)
        file_title = file_title + f"_RID_{sameName_Key}"
    titleList.append(file_title)
    file_path = os.path.join(folderPath, f"{file_title}.txt")


    with open(file_path , 'wb') as f:
        f.write(bookInfo.content)
        
    print(f"已下載 {file_title}")


books = GetAllBooksURL()
for b in books:
    getBooksInfo(b)